In [1]:
import gym
import gym_kraby
import math as maths
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
from copy import deepcopy as DC
import torch
clear_output()

In [2]:
class hexapod:
    def __init__(self):
        self.env= gym.make('gym_kraby:HexapodBulletEnv-v0', render=True)
        obs=self.env.reset()
        self.action_space=self.env.action_space.sample()  # take a random action
        self.sensors={"foot_0":4,"endcap_0":5,"foot_1":10,"endcap_1":11,
            "foot_2":16,"endcap_2": 17,"foot_3": 22,"endcap_3": 23,
            "foot_4": 28,"endcap_4": 29,"foot_5": 34,"endcap_5":35}
    def list_feet(self):
        _link_name_to_index = {self.env.p.getBodyInfo(self.env.robot_id)[0].decode('UTF-8'):-1,}
        for _id in range(self.env.p.getNumJoints(self.env.robot_id)):
            _name = self.env.p.getJointInfo(self.env.robot_id, _id)[12].decode('UTF-8')
            _link_name_to_index[_name] = _id
            if "endcap" in _name or "foot" in _name:
                print(_name,_id)
    def reset(self):
        obs=self.env.reset()
    def step(self,a):
        """
        step through environment with action a
        return reward and the force on the feet that are contacted numbered 0-5 for each leg
        """
        observation, reward, done, _ = self.env.step(a)  # step
        filtered=self.getContact()
        return reward,filtered
    def getAngles(self):
        a=[self.getAngle(i) for i in range(18)]
        return np.array(a)
    def getContact(self):
        VALS=np.array([[5,4],[11,10],[17,16],[23,22],[29,28],[35,34]])
        contact=self.env.p.getContactPoints(bodyA=self.env.robot_id)
        a_=np.array([[5,0],[11,0],[17,0],[23,0],[29,0],[35,0]])
        for c in contact:
            p=c[3]
            f=c[9]
            if p in VALS.flatten():
                id=np.where(VALS == p)
                id=[id[0],id[1]][0]
                if a_[id[0]][0]==p: a_[id[0]][1]=f
                elif a_[id[0]][1]<f: #largest force saves
                    a_[id[0]][0]=p
                    a_[id[0]][1]=f
        return a_
    def getAngle(self,num):
        JOINT=[1,2,4,7,8,10,13,14,16,19,20,22,25,26,28,31,32,34]
        b=self.env.p.getJointState(self.env.robot_id,JOINT[num])
        d=b[0]#convert to degrees
        return (maths.pi/2)+d
    def getImage(self):
        img = self.env.p.getCameraImage(224, 224, renderer=self.env.p.ER_BULLET_HARDWARE_OPENGL)
        return img[2]
    def close(self):
        self.env.close()
    def moveAngles(self,ang):
        for j in range(5): #for accuracy do multiple times
            mov=np.zeros_like(ang)
            angles=self.getAngles()
            for i in range(len(mov)):
                mov[i]=self.moveToAngle(ang[i],angles[i])
            self.step(mov)
    def moveToAngle(self,start,to):
        end=start-to
        k=1/np.deg2rad(18)
        return end*k
    def place(self,legNo,ANG1,ANG2=None):
        legNo=legNo*3
        act=self.getAngles()
        act[legNo+1]=ANG1
        ang=(np.pi)-ANG1
        act[legNo+2]=ang
        if ANG2!=None: act[legNo]=ANG2
        self.moveAngles(act)
    def moveThrough(self,ANGLES):
        for i in range(6):
            self.place(i,ANGLES[:,0][i],ANG2=ANGLES[:,1][i])
    

In [3]:
robot=hexapod()
act=np.ones_like(robot.action_space)*np.deg2rad(80)
robot.moveAngles(act)
robot.place(5,np.deg2rad(120))

c:\Users\dexte\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\utils\passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
c:\Users\dexte\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\utils\passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
c:\Users\dexte\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\utils\passive_env_checker.py:195: UserWarning: WARN: The result returned by `env.reset()` was not a tuple of the form `(obs, info)`, where `obs` is a observation and `info` is a dictionary containing additional information. Actual type: `<class 'numpy.ndarray'>`
  logger.warn(
c:\Users\dexte\AppData\Local\Programs\Python\Pyt

In [4]:
def place(legNo,ANG1,ANG2=None):
        legNo=legNo*3
        act=robot.getAngles()
        act[legNo+1]=ANG1
        ang=(np.pi)-ANG1
        act[legNo+2]=ang
        if ANG2!=None: act[legNo]=ANG2
        robot.moveAngles(act)
place(1,np.deg2rad(180))

## Evolving patterns

### Model

In [5]:
class Agent:
    def __init__(self, num_input, layers, num_output):
        assert type(layers)==type([]), "Error with layers, give array of the number of layers"
        self.num_input = num_input  #set input number
        self.num_output = num_output #set ooutput number
        self.hidden=[]
        last=num_input
        self.num_genes=0
        for layer in layers:
            self.hidden.append(layer)
            self.num_genes+=(last * layer)
            last=layer
        self.num_genes +=(self.hidden[-1]*num_output)+num_output
        self.weights = None
        self.hidden_weights=None
        self.bias = None
    def set_genes(self, gene):
        weight_idxs = self.num_input * self.hidden[0] #size of weights to hidden
        current=weight_idxs
        weights_idxs=[current] #start with end of last
        for i in range(len(self.hidden)-1):
            current+=self.hidden[i]*self.hidden[i+1] #calculate next idx for each layer
            weights_idxs.append(current)
        bias_idxs=None
        weights_idxs.append(self.hidden[-1] * self.num_output + weights_idxs[-1]) #add last layer heading to output
        bias_idxs = weights_idxs[-1]+ self.num_output #sizes of biases
        w = gene[0 : weight_idxs].reshape(self.hidden[0], self.num_input)   #merge genes
        ws=[]
        for i in range(len(self.hidden)-1):
            ws.append(gene[weights_idxs[i] : weights_idxs[i+1]].reshape(self.hidden[i+1], self.hidden[i]))
        ws.append(gene[weights_idxs[-2] : weights_idxs[-1]].reshape(self.num_output, self.hidden[-1]))
        b = gene[weights_idxs[-1]: bias_idxs].reshape(self.num_output,) #merge genes

        self.weights = torch.from_numpy(w) #assign weights
        self.hidden_weights=[]
        for w in ws:
            self.hidden_weights.append(torch.from_numpy(w))
        self.bias = torch.from_numpy(b) #assign biases

    def forward(self, x):
        x=x.flatten()
        x=torch.tensor(x[:,np.newaxis]).float() 
        #x = torch.tensor(np.dot(self.weights.float(),x).flatten()).float()
        #run through forward layers
        x = torch.mm(x.T, self.weights.T.float()) #first layer

        for i in range(len(self.hidden_weights)-1):
            x = torch.mm(x,self.hidden_weights[i].T.float()) #second layer
        x=torch.sigmoid(x)
        return (torch.mm(x,self.hidden_weights[-1].T.float()) + self.bias).detach().numpy() #third layer




### GA

In [11]:
def mutate(geno):
    geno+=np.random.normal(0,3,(geno.shape))
    geno[geno>12]=12
    geno[geno<-12]=-12
    return 

def gen_genotype(shape): #generate a genotype of 1 and 0
    return np.random.normal(0,5,(shape))

def gen_pop(size,pergeno):
    return gen_genotype(size*pergeno).reshape((size,pergeno))

def fitness(robot, start, energy_consumption):
    # Define weights for each fitness component
    cubePos, end_orientation = robot.env.p.getBasePositionAndOrientation(robot.env.robot_id)
    distance_travelled=np.linalg.norm(np.array(start)[0:2]-np.array(cubePos)[0:2])
    distance_weight = 0.6
    energy_weight = 0.3
    orientation_weight = 0.1
    
    # Normalize the orientation values to be between -1 and 1
    orientation_norm = maths.sqrt(end_orientation[0] ** 2 + end_orientation[1] ** 2)
    orientation_norm = max(0.001, orientation_norm)  # Prevent division by zero
    orientation_norm = min(1, orientation_norm)
    orientation_norm = (2 / maths.pi) * maths.atan(orientation_norm)
    
    # Calculate the overall fitness value as a weighted sum of the components
    fitness = (distance_weight * distance_travelled) + (energy_weight * (1 / energy_consumption)) + (orientation_weight * orientation_norm)
    if np.inf == fitness:
        return 0
    return fitness
def getFallen():
    cubePos, cubeOrn = robot.env.p.getBasePositionAndOrientation(robot.env.robot_id)
    if cubePos[-1]<0: return True
    return False
def runTrial(agent,robot,T):
    dt=0.02
    robot.reset()
    done=False
    t=0
    fx=0
    pattern=np.zeros((12))
    start, cubeOrn = robot.env.p.getBasePositionAndOrientation(robot.env.robot_id)
    while t<T and not done:
        """r,c=robot.step(pattern)
        f=c[:,1] #get forces"""
        #angles=robot.getAngles().reshape(6,3)
        pattern=np.tanh(agent.forward(pattern))
        angles=pattern.reshape(6,2)
        legVert=angles[:,0]
        legHorz=angles[:,1]
        #convert to standing
        angles[:,0][angles[:,0]>0]=np.deg2rad(120)
        angles[:,0][angles[:,0]<=0]=np.deg2rad(80)
        #convert to swing
        angles[:,1][angles[:,1]>0]=np.deg2rad(120)
        angles[:,1][angles[:,1]<=0]=np.deg2rad(40)
        robot.moveThrough(angles) #move robot
        fx+=np.sum(np.abs(pattern))/(len(pattern)*10)
        
        t+=dt
    #f=fitness(robot,start)
    f_=fitness(robot,start,fx)
    return f_


def microbial(agent,robot,population,generations=500,T=50):
    pop_size=len(population)
    overTime=np.zeros((generations,))
    fitness_matrix=np.zeros((pop_size))
    for gen in range(generations): #run trial
        clear_output()
        print("Generation:",gen,"Fitness",overTime[max(gen-1,0)])
        n1=np.random.randint(0,pop_size-1)
        n2=n1+1
        if n2==pop_size: n2=n1-1
        #while n1==n2: n2=np.random.randint(0,pop_size-1) #make sure it is not the same
        g1=DC(population[n1])
        g2=DC(population[n2])
        #get fitnesses
        agent.set_genes(g1)
        f1=0
        for j in range(3):
            r=runTrial(agent,robot,T)
            f1+=r
        f1/=3
        agent.set_genes(g1)
        f2=0
        for j in range(3):
            r=runTrial(agent,robot,T)
            f2+=r
        f2/=3
        fitness_matrix[n1]=f1
        fitness_matrix[n2]=f2
        #tournament
        if f1>f2:
            population[n2]=DC(mutate(DC(g1)))
        elif f2>f1:
            population[n1]=DC(mutate(DC(g2)))
        overTime[gen]=np.max(fitness_matrix)
    return overTime,fitness_matrix

In [12]:
agent=Agent(12,[20,20],12)
population=gen_pop(50,agent.num_genes) #create population
try: population=np.load("model_kine") #if exists
except: pass

genotype=np.random.normal(0,5,(agent.num_genes,))
agent.set_genes(population[0])
microbial(agent,robot,population,generations=500,T=2)

Generation: 0 Fitness 0.0
